## Compute driving distances - IN PROGRESS
Here we modify our previous analysis that computed the Euclidean distance between candidate site (exits) and existing infrastructure (DCFC) by instead measuring *driving* distances. To do this we need a network dataset, often called a *graph dataset* of roads, which we get from OpenStreetMap, and a means for analyzing network datasets, which we can do via the `osmnx` package. 

The analysis here is guided by the analysis presented in [Lesson 6: Network Analysis in Python](https://automating-gis-processes.github.io/site/notebooks/L6/network-analysis.html) of the Automating GIS curriculum.

Prior to running this notebook, you should have already run `A4-Fetch-NC-roads-as-graph.ipynb` to download the NC roads as a graphml dataset into your Data/OSM folder.

The workflow here is:
* Import major roads as a graph dataset from OSM saved graphml file
* Import the DCFC csv file and convert to a geopandas dataframe
* Import the exits feature class as a geopdandas dataframe
* For each exit:
 * Find the nearest node in the NC graph
 * Compute the subgraph of edges within 100 miles of the node; extract to a shape, and add to a list
 * Compute the subgraph of edges within 50 miles of the node; extract to a shape, and add to a list
* Union all 100 mile drive zones
* Union all 50 mile drive zones
* Eliminate all 50 mile drive zones from the 100 mile drive zones
* Identify all 

In [ ]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [ ]:
#Load in the NC road network
nc_graph = ox.load_graphml('NC_roads.graphml',folder='../Data/OSM/')

In [ ]:
#Load in the DCFC locations as pandas dataframe
dcfc_df = pd.read_csv('../Data/NREL/DCFC.csv')

In [ ]:
#Convert to a geopandas dataframe
geom_points = [Point(xy) for xy in zip(dcfc_df['longitude'],dcfc_df['latitude'])]
dcfc_gdf = gpd.GeoDataFrame(dcfc_df,geometry=geom_points,crs=4326)

In [ ]:
#Compute areas within 50 miles of a point

#Get the first point, as an example
thePoint = dcfc_gdf.at[0,'geometry']

https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.geo_utils.get_nearest_node

In [ ]:
#Convert the shapely Point to a tuple (y,x)
yx = (thePoint.y,thePoint.x)

In [ ]:
#Get the graph node nearest the point
theStartNode = ox.get_nearest_node(nc_graph,yx)
theStartNode

https://github.com/gboeing/osmnx-examples/blob/master/notebooks/13-isolines-isochrones.ipynb

In [ ]:
#Compute the subgraph: all points within X miles of theStartNode
subgraph_50 = nx.ego_graph(nc_graph,theStartNode,radius=1609.34 * 50,distance='length')
subgraph_100 = nx.ego_graph(nc_graph,theStartNode,radius=1609.34 * 100,distance='length')

In [ ]:
#Convert the subgraph to geodataframes of nodes and edges
node50_gdf,edge50_gdf = ox.graph_to_gdfs(subgraph_50)
node100_gdf,edge100_gdf = ox.graph_to_gdfs(subgraph_100)

In [ ]:
drive50 = edge50_gdf['geometry'].unary_union
type(drive50)

In [ ]:
drive100 = edge100_gdf['geometry'].unary_union
type(drive100)

In [ ]:
dcfc_gdf.at[0,'drive50'] = drive50
dcfc_gdf.at[0,'drive100'] = drive100

In [ ]:
#Create a function that extracts areas within 100 

In [ ]:
#Find the symmetric difference between the two edge geodataframes
edge_gdf = gpd.overlay(edge100_gdf,edge50_gdf,how='difference')

In [ ]:
ax = edge100_gdf.plot(figsize=(10,10),color='grey',linewidth=15)
edge_gdf.plot(ax=ax, color='yellow',alpha=1,linewidth=1)
edge50_gdf.plot(ax=ax, color='blue',alpha=0.1,linewidth=0.1)

In [ ]:
#Get all roads within 50 miles
sub50 = nx.ego_graph(nc_graph,nearNodes,radius=1609.34 * 50,distance='length')

In [ ]:
def get_start_node(theShape):
    yx = (theShape.y, theShape.x)
    theStartNode = ox.get_nearest_nodes

In [ ]:
dcfc_gdf['start_id'] = dcfc_gdf['geometry'].apply(ox.get_nearest_node)